In [72]:
import requests
import credentials

In [73]:
session = requests.Session()

In [74]:
instrument = "EUR_USD"
count = 10
granularity = "H1"

In [75]:
# this pattern is taken from the documentation of oanda site and it is better to look for it yourself
url = f"{credentials.OANDA_URL}/instruments/{instrument}/candles"

In [76]:
url

'https://api-fxpractice.oanda.com/v3/instruments/EUR_USD/candles'

In [77]:
param = dict(
    count=count,
    granularity=granularity,
    price="MBA"
)

In [78]:
param

{'count': 10, 'granularity': 'H1', 'price': 'MBA'}

In [79]:
API_KEY = "388d8d5440ad979dd6e1745b1f566bde-a5dea5109879fe5f64d0c8b871ab59e8"
header = f"Bearer {API_KEY}"
response = session.get(url, params=param, headers=credentials.SECURE_HEADER)

In [81]:
response.status_code

200

In [82]:
data = response.json()

In [83]:
data.keys()

dict_keys(['instrument', 'granularity', 'candles'])

In [84]:
len(data['candles'])

10

In [86]:
for candle in data['candles']:
    print(candle)

{'complete': True, 'volume': 6924, 'time': '2022-06-17T11:00:00.000000000Z', 'bid': {'o': '1.05194', 'h': '1.05227', 'l': '1.04828', 'c': '1.04852'}, 'mid': {'o': '1.05202', 'h': '1.05234', 'l': '1.04836', 'c': '1.04859'}, 'ask': {'o': '1.05210', 'h': '1.05241', 'l': '1.04843', 'c': '1.04866'}}
{'complete': True, 'volume': 8251, 'time': '2022-06-17T12:00:00.000000000Z', 'bid': {'o': '1.04849', 'h': '1.05074', 'l': '1.04750', 'c': '1.04941'}, 'mid': {'o': '1.04856', 'h': '1.05082', 'l': '1.04758', 'c': '1.04948'}, 'ask': {'o': '1.04863', 'h': '1.05090', 'l': '1.04765', 'c': '1.04955'}}
{'complete': True, 'volume': 9713, 'time': '2022-06-17T13:00:00.000000000Z', 'bid': {'o': '1.04941', 'h': '1.04995', 'l': '1.04732', 'c': '1.04836'}, 'mid': {'o': '1.04948', 'h': '1.05002', 'l': '1.04740', 'c': '1.04844'}, 'ask': {'o': '1.04954', 'h': '1.05009', 'l': '1.04747', 'c': '1.04851'}}
{'complete': True, 'volume': 12136, 'time': '2022-06-17T14:00:00.000000000Z', 'bid': {'o': '1.04838', 'h': '1.04

In [87]:
prices = ['mid', 'bid', 'ask']
ohlc = ['o', 'h', 'l', 'c']

In [91]:
# this is to understand the logic of what we're going to do later
data['candles'][0]['bid']['o']

'1.05194'

In [88]:
for price in prices:
    for oh in ohlc:
        print(f"{price}-{oh}")

mid-o
mid-h
mid-l
mid-c
bid-o
bid-h
bid-l
bid-c
ask-o
ask-h
ask-l
ask-c


In [93]:
our_data = []
for candle in data['candles']:
    if candle['complete'] == False:
        continue
    new_dict = {}
    new_dict['time'] = candle['time']
    new_dict['volume'] = candle['volume']
    for price in prices:
        for oh in ohlc:
            new_dict[f"{price}_{oh}"] = candle[price][oh]
    our_data.append(new_dict)
print(our_data[0])

{'time': '2022-06-17T11:00:00.000000000Z', 'volume': 6924, 'mid_o': '1.05202', 'mid_h': '1.05234', 'mid_l': '1.04836', 'mid_c': '1.04859', 'bid_o': '1.05194', 'bid_h': '1.05227', 'bid_l': '1.04828', 'bid_c': '1.04852', 'ask_o': '1.05210', 'ask_h': '1.05241', 'ask_l': '1.04843', 'ask_c': '1.04866'}


In [94]:
import pandas as pd

In [96]:
candles_dataframe = pd.DataFrame.from_dict(our_data)
candles_dataframe

,time,volume,mid_o,mid_h,mid_l,mid_c,bid_o,bid_h,bid_l,bid_c,ask_o,ask_h,ask_l,ask_c
0,2022-06-17T11:00:00.000000000Z,6924,1.05202,1.05234,1.04836,1.04859,1.05194,1.05227,1.04828,1.04852,1.05210,1.05241,1.04843,1.04866
1,2022-06-17T12:00:00.000000000Z,8251,1.04856,1.05082,1.04758,1.04948,1.04849,1.05074,1.04750,1.04941,1.04863,1.05090,1.04765,1.04955
2,2022-06-17T13:00:00.000000000Z,9713,1.04948,1.05002,1.04740,1.04844,1.04941,1.04995,1.04732,1.04836,1.04954,1.05009,1.04747,1.04851
3,2022-06-17T14:00:00.000000000Z,12136,1.04844,1.04872,1.04445,1.04584,1.04838,1.04865,1.04438,1.04577,1.04850,1.04880,1.04452,1.04590
4,2022-06-17T15:00:00.000000000Z,8796,1.04588,1.04776,1.04550,1.04735,1.04581,1.04769,1.04543,1.04728,1.04594,1.04784,1.04557,1.04742
5,2022-06-17T16:00:00.000000000Z,5814,1.04736,1.04764,1.04649,1.04676,1.04729,1.04757,1.04642,1.04670,1.04743,1.04771,1.04656,1.04683
6,2022-06-17T17:00:00.000000000Z,5485,1.04674,1.04998,1.04666,1.04906,1.04668,1.04991,1.04660,1.04899,1.04681,1.05005,1.04673,1.04914
7,2022-06-17T18:00:00.000000000Z,5171,1.04910,1.05028,1.04902,1.04912,1.04902,1.05020,1.04893,1.04904,1.04919,1.05036,1.04911,1.04920
8,2022-06-17T19:00:00.000000000Z,4884,1.04914,1.04978,1.04880,1.04948,1.04907,1.04970,1.04871,1.04939,1.04922,1.04987,1.04887,1.04956
9,2022-06-17T20:00:00.000000000Z,1965,1.04945,1.05003,1.04896,1.04938,1.04937,1.04994,1.04886,1.04910,1.04953,1.05016,1.04905,1.04966


In [97]:
candles_dataframe.to_pickle('EUR_USD_H1.pkl')

In [98]:
df = pd.read_pickle('EUR_USD_H1.pkl')

In [99]:
df

,time,volume,mid_o,mid_h,mid_l,mid_c,bid_o,bid_h,bid_l,bid_c,ask_o,ask_h,ask_l,ask_c
0,2022-06-17T11:00:00.000000000Z,6924,1.05202,1.05234,1.04836,1.04859,1.05194,1.05227,1.04828,1.04852,1.05210,1.05241,1.04843,1.04866
1,2022-06-17T12:00:00.000000000Z,8251,1.04856,1.05082,1.04758,1.04948,1.04849,1.05074,1.04750,1.04941,1.04863,1.05090,1.04765,1.04955
2,2022-06-17T13:00:00.000000000Z,9713,1.04948,1.05002,1.04740,1.04844,1.04941,1.04995,1.04732,1.04836,1.04954,1.05009,1.04747,1.04851
3,2022-06-17T14:00:00.000000000Z,12136,1.04844,1.04872,1.04445,1.04584,1.04838,1.04865,1.04438,1.04577,1.04850,1.04880,1.04452,1.04590
4,2022-06-17T15:00:00.000000000Z,8796,1.04588,1.04776,1.04550,1.04735,1.04581,1.04769,1.04543,1.04728,1.04594,1.04784,1.04557,1.04742
5,2022-06-17T16:00:00.000000000Z,5814,1.04736,1.04764,1.04649,1.04676,1.04729,1.04757,1.04642,1.04670,1.04743,1.04771,1.04656,1.04683
6,2022-06-17T17:00:00.000000000Z,5485,1.04674,1.04998,1.04666,1.04906,1.04668,1.04991,1.04660,1.04899,1.04681,1.05005,1.04673,1.04914
7,2022-06-17T18:00:00.000000000Z,5171,1.04910,1.05028,1.04902,1.04912,1.04902,1.05020,1.04893,1.04904,1.04919,1.05036,1.04911,1.04920
8,2022-06-17T19:00:00.000000000Z,4884,1.04914,1.04978,1.04880,1.04948,1.04907,1.04970,1.04871,1.04939,1.04922,1.04987,1.04887,1.04956
9,2022-06-17T20:00:00.000000000Z,1965,1.04945,1.05003,1.04896,1.04938,1.04937,1.04994,1.04886,1.04910,1.04953,1.05016,1.04905,1.04966
